<a href="https://colab.research.google.com/github/isurushanaka/AII_Course-1/blob/main/Neural%20Networks/6_pre-trained_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cats and Dogs Classification

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model

#### Download and Extract the Dataset
[Dataset Link](https://www.microsoft.com/en-us/download/details.aspx?id=54765)

In [ ]:
path_to_dataset = "PetImages"

In [ ]:
# Define the input shape
input_shape = (150, 150, 3)

In [ ]:
# Load the pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 12s 0us/step


In [ ]:
# Create a new top classification layer
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

# Build the modified model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Load and preprocess the training and validation data
train_generator = train_datagen.flow_from_directory(path_to_dataset,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')


Found 25000 images belonging to 2 classes.


In [ ]:
# Train the model
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=10,
          validation_data=validation_generator,
          validation_steps=len(validation_generator))

In [ ]:
# Compute the training accuracy
accuracy = model.evaluate(train_generator)[1]
print('Accuracy:', accuracy)

In [ ]:
# Save the trained model
model.save('cat_dog_classifier.h5')

In [ ]:
import matplotlib.pyplot as plt
# Plot some example images
class_names = ['Cat', 'Dog']
example_images = train_generator.next()[0]
example_labels = train_generator.next()[1]

fig, axes = plt.subplots(2, 5, figsize=(12, 6))
axes = axes.ravel()

for i in range(10):
    axes[i].imshow(example_images[i])
    axes[i].set_title(class_names[int(example_labels[i])])
    axes[i].axis('off')

plt.tight_layout()
plt.show()